In [334]:
import pandas as pd
import numpy as np
import yfinance as yf

from talib import abstract
# directly import sma
SMA = abstract.SMA

from backtesting import Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
from backtesting import Backtest

## Getting Data from Yahoo Example
Additional info: https://pypi.org/project/yfinance/

In [336]:
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,3y,5y,10y,ytd,max
# S&P500 index: ^GSPC
stock = yf.Ticker("SPY")

# one way to get data for period of interest
t1y = stock.history(period="1y")
t1y

# Other potential fields of interest
# stock.info
# stock.major_holders
# stock.recommendations
# stock.options

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-02-10,325.157759,328.613219,325.118484,328.544495,42070000,0.0,0
2020-02-11,329.997356,330.841576,328.544477,329.113861,54864500,0.0,0
2020-02-12,330.655073,331.460048,330.262412,331.234283,43992700,0.0,0
2020-02-13,329.702846,331.921425,329.408358,330.880859,54501900,0.0,0
2020-02-14,331.322634,331.538602,330.036652,331.410980,64582200,0.0,0
...,...,...,...,...,...,...,...
2021-02-03,382.440002,383.700012,380.480011,381.850006,52427100,0.0,0
2021-02-04,382.959991,386.239990,381.970001,386.190002,47142600,0.0,0
2021-02-05,388.200012,388.470001,386.140015,387.709991,48620300,0.0,0


## Strategies
More info: https://mrjbq7.github.io/ta-lib/func_groups/momentum_indicators.html

### SMA Cross Strategy

In [368]:
class SmaCross(Strategy):
    # Define the a MA lags as *class variables*
    n1 = 3
    n2 = 15
    
    def init(self):
        # Precompute the moving averages
        self.ma3 = self.I(SMA, self.data.Close, self.n1)
        self.ma15 = self.I(SMA, self.data.Close, self.n2)
            
    def next(self):
        # If Closing price crosses above sma
        if crossover(self.ma3, self.ma15):
            #self.position.close()
            self.buy()

        # Else, if Low price crosses below sma
        elif crossover(self.ma15, self.ma3):
            self.position.close()
            #self.sell() # long-only

### SMA Cross Strategy

## Backtesting Strategies
Additional info: https://kernc.github.io/backtesting.py/doc/examples/Quick%20Start%20User%20Guide.html

In [366]:
# Specify data timeframe and strategy to test
# Get data from Yahoo
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,3y,5y,10y,ytd,max
# S&P500 index: ^GSPC
data = yf.download("SPY", period="1y")
strategy = SmaCross

# Run backtesting
bt = Backtest(data, strategy, cash=10_000, commission=0)
stats = bt.run()

# Display stats and plot results
print(stats)
bt.plot()

[*********************100%***********************]  1 of 1 completed
Start                     2020-02-10 00:00:00
End                       2021-02-09 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   70.355731
Equity Final [$]                 12828.098282
Equity Peak [$]                  12904.378799
Return [%]                          28.280983
Buy & Hold Return [%]               16.603923
Return (Ann.) [%]                   28.154765
Volatility (Ann.) [%]               23.141767
Sharpe Ratio                         1.216621
Sortino Ratio                        2.335032
Calmar Ratio                         2.802646
Max. Drawdown [%]                  -10.045781
Avg. Drawdown [%]                    -2.60921
Max. Drawdown Duration       90 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    8
Win Rate [%]                             87.5
Best Trade [%]                      10.430191
Worst Trade

In [342]:
# Contains individual trade data
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,39,34,67,255.699997,282.369995,1040.129929,0.104302,2020-03-30,2020-05-15,46 days
1,37,69,87,294.350006,298.019989,135.789368,0.012468,2020-05-19,2020-06-15,27 days
2,35,91,92,314.170013,307.989990,-216.300812,-0.019671,2020-06-19,2020-06-22,3 days
3,34,100,147,314.239990,337.549988,792.539917,0.074179,2020-07-02,2020-09-09,69 days
4,34,163,178,337.690002,342.959991,179.179626,0.015606,2020-10-01,2020-10-22,21 days
5,34,189,221,349.929993,368.279999,623.900208,0.052439,2020-11-06,2020-12-23,47 days
6,33,224,245,373.809998,375.630005,60.060242,0.004869,2020-12-29,2021-01-29,31 days
7,32,249,252,382.959991,389.609985,212.799805,0.017365,2021-02-04,2021-02-09,5 days


In [341]:
# Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.
stats['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2020-02-10,10000.000000,0.000000,NaT
2020-02-11,10000.000000,0.000000,NaT
2020-02-12,10000.000000,0.000000,NaT
2020-02-13,10000.000000,0.000000,NaT
2020-02-14,10000.000000,0.000000,NaT
...,...,...,...
2021-02-03,12615.298477,0.022402,NaT
2021-02-04,12718.658829,0.014392,NaT
2021-02-05,12767.298477,0.010623,NaT
2021-02-08,12856.899063,0.003679,NaT
